In [1]:

!pip install -r requirements.txt

In [2]:
# kg baglantısı kur(neo4j clientı olustur) amac verileri py ile ekleyebilmek
# llm embedding clientını kur
# uretilen sentetik verileri node olarak basmak(once eski semayi bas)
# node olarak basarken textlerini(yani tarif detayını) name kısmına göre embedding alıp tarif detayına embedding attribute'unu eklemek
# son semayı verip system prompt olarak llm e vermek cypher query uretmek

In [3]:
import os
import getpass
from dotenv import load_dotenv

from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import GraphCypherQAChain # Doğru import yolu GraphCypherQAChain için
# .env dosyasındaki ortam değişkenlerini yükle
load_dotenv()

True

In [4]:
# --- 1. API Anahtarları ve Neo4j Bağlantı Bilgileri ---
def get_api_keys():
    """OpenAI API anahtarını ve Neo4j bilgilerini alır."""
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        openai_api_key = getpass.getpass("OpenAI API Anahtarınızı Girin: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

    neo4j_uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
    neo4j_username = os.getenv("NEO4J_USERNAME", "neo4j")
    neo4j_password = os.getenv("NEO4J_PASSWORD")
    if not neo4j_password:
        neo4j_password = getpass.getpass("Neo4j Şifrenizi Girin: ")
    
    return neo4j_uri, neo4j_username, neo4j_password

In [11]:
# --- 2. Neo4j ve Embedding İstemcisi Kurulumu ---
def setup_clients(neo4j_uri, neo4j_username, neo4j_password):
    """Neo4j Graph ve Embedding istemcisini kurar."""
    graph = Neo4jGraph(
        url=neo4j_uri,
        username=neo4j_username,
        password=neo4j_password
    )
    
    # embeddings_client = OpenAIEmbeddings(model="text-embedding-3-small")
    # llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-nano-2025-04-14") # veya gpt-3.5-turbo
    # return graph, embeddings_client, llm


    from sentence_transformers import SentenceTransformer
    # OpenAI yerine SentenceTransformer kullanıyoruz
    embeddings_client = SentenceTransformer("all-MiniLM-L6-v2")
    # Eğer LLM kullanacaksan, bu kısmı isteğe göre ayarlayabilirsin
    llm =  ChatOpenAI(temperature=0, model_name="gpt-4.1-nano-2025-04-14")  # ya da ChatOpenAI(...) ile devam edebilirsin (OpenAI quota uygunsa)
    return graph, embeddings_client, llm

In [12]:
# --- 3. Başlangıç Şemasını Neo4j'e Basma ---
def populate_initial_schema(graph):
    """Kullanıcının verdiği başlangıç şemasını Neo4j'e yazar."""
    initial_cypher_statements = [
        # MALZEMELER
        'MERGE (Sugar:Ingredient {name: "Şeker"}) ON CREATE SET Sugar.category = "Tatlandırıcı";',
        'MERGE (Milk:Ingredient {name: "Süt"}) ON CREATE SET Milk.category = "Süt Ürünü";',
        'MERGE (Egg:Ingredient {name: "Yumurta"}) ON CREATE SET Egg.category = "Hayvansal";',
        'MERGE (Flour:Ingredient {name: "Un"}) ON CREATE SET Flour.category = "Tahıl";',
        'MERGE (Butter:Ingredient {name: "Tereyağı"}) ON CREATE SET Butter.category = "Süt Ürünü";',
        'MERGE (Vanilla:Ingredient {name: "Vanilin"}) ON CREATE SET Vanilla.category = "Aroma";',
        'MERGE (Cream:Ingredient {name: "Krema"}) ON CREATE SET Cream.category = "Süt Ürünü";',
        'MERGE (Chocolate:Ingredient {name: "Çikolata"}) ON CREATE SET Chocolate.category = "Tatlandırıcı";',
        'MERGE (Gelatin:Ingredient {name: "Jelatin"}) ON CREATE SET Gelatin.category = "Katılaştırıcı";',
        'MERGE (Honey:Ingredient {name: "Bal"}) ON CREATE SET Honey.category = "Doğal Tatlandırıcı";',
        'MERGE (Biscuit:Ingredient {name: "Bisküvi"}) ON CREATE SET Biscuit.category = "Hazır Gıda";',
        'MERGE (Cinnamon:Ingredient {name: "Tarçın"}) ON CREATE SET Cinnamon.category = "Baharat";',
        'MERGE (Rice:Ingredient {name: "Pirinç"}) ON CREATE SET Rice.category = "Tahıl";',
        # LEZZETLER
        'MERGE (Sweet:Flavor {name: "Tatlı"});',
        'MERGE (Creamy:Flavor {name: "Kremamsı"});',
        'MERGE (Caramel:Flavor {name: "Karamelli"});',
        # TEKNİKLER
        'MERGE (Bake:Technique {name: "Fırınlama"});',
        'MERGE (Boil:Technique {name: "Kaynatma"});',
        'MERGE (Mix:Technique {name: "Karıştırma"});',
        'MERGE (Chill:Technique {name: "Soğutma"});',
        # TARİFLER
        'MERGE (Pudding:Recipe {name: "Sütlaç"}) ON CREATE SET Pudding.difficulty = "Kolay", Pudding.time_minutes = 45;',
        'MERGE (Cake:Recipe {name: "Kek"}) ON CREATE SET Cake.difficulty = "Orta", Cake.time_minutes = 60;',
        'MERGE (Cheesecake:Recipe {name: "Cheesecake"}) ON CREATE SET Cheesecake.difficulty = "Zor", Cheesecake.time_minutes = 90;',
        'MERGE (Truffle:Recipe {name: "Çikolata Truffle"}) ON CREATE SET Truffle.difficulty = "Kolay", Truffle.time_minutes = 30;',
        'MERGE (Magnolia:Recipe {name: "Magnolia"}) ON CREATE SET Magnolia.difficulty = "Orta", Magnolia.time_minutes = 50;',
        # MALZEME-İÇERİK İLİŞKİLERİ
        'MATCH (r:Recipe {name: "Sütlaç"}), (i:Ingredient {name: "Şeker"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Sütlaç"}), (i:Ingredient {name: "Süt"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Sütlaç"}), (i:Ingredient {name: "Vanilin"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Sütlaç"}), (i:Ingredient {name: "Pirinç"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Kek"}), (i:Ingredient {name: "Un"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Kek"}), (i:Ingredient {name: "Yumurta"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Kek"}), (i:Ingredient {name: "Şeker"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Kek"}), (i:Ingredient {name: "Tereyağı"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Kek"}), (i:Ingredient {name: "Vanilin"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (i:Ingredient {name: "Krema"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (i:Ingredient {name: "Bisküvi"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (i:Ingredient {name: "Tereyağı"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (i:Ingredient {name: "Şeker"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Çikolata Truffle"}), (i:Ingredient {name: "Çikolata"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Çikolata Truffle"}), (i:Ingredient {name: "Krema"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Çikolata Truffle"}), (i:Ingredient {name: "Tereyağı"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Magnolia"}), (i:Ingredient {name: "Süt"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Magnolia"}), (i:Ingredient {name: "Un"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Magnolia"}), (i:Ingredient {name: "Şeker"}) MERGE (r)-[:CONTAINS]->(i);',
        'MATCH (r:Recipe {name: "Magnolia"}), (i:Ingredient {name: "Tarçın"}) MERGE (r)-[:CONTAINS]->(i);',
        # LEZZET İLİŞKİLERİ
        'MATCH (r:Recipe {name: "Sütlaç"}), (f:Flavor {name: "Tatlı"}) MERGE (r)-[:HAS_FLAVOR]->(f);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (f:Flavor {name: "Kremamsı"}) MERGE (r)-[:HAS_FLAVOR]->(f);',
        'MATCH (r:Recipe {name: "Çikolata Truffle"}), (f:Flavor {name: "Karamelli"}) MERGE (r)-[:HAS_FLAVOR]->(f);',
        'MATCH (r:Recipe {name: "Kek"}), (f:Flavor {name: "Tatlı"}) MERGE (r)-[:HAS_FLAVOR]->(f);',
        'MATCH (r:Recipe {name: "Magnolia"}), (f:Flavor {name: "Kremamsı"}) MERGE (r)-[:HAS_FLAVOR]->(f);',
        # TEKNİK İLİŞKİLERİ
        'MATCH (r:Recipe {name: "Sütlaç"}), (t:Technique {name: "Kaynatma"}) MERGE (r)-[:USES_TECHNIQUE]->(t);',
        'MATCH (r:Recipe {name: "Kek"}), (t:Technique {name: "Fırınlama"}) MERGE (r)-[:USES_TECHNIQUE]->(t);',
        'MATCH (r:Recipe {name: "Cheesecake"}), (t:Technique {name: "Fırınlama"}) MERGE (r)-[:USES_TECHNIQUE]->(t);',
        'MATCH (r:Recipe {name: "Çikolata Truffle"}), (t:Technique {name: "Karıştırma"}) MERGE (r)-[:USES_TECHNIQUE]->(t);',
        'MATCH (r:Recipe {name: "Magnolia"}), (t:Technique {name: "Soğutma"}) MERGE (r)-[:USES_TECHNIQUE]->(t);',
        # YERİNE KULLANILABİLİR MALZEME İLİŞKİLERİ
        'MATCH (i1:Ingredient {name: "Bal"}), (i2:Ingredient {name: "Şeker"}) MERGE (i1)-[:CAN_REPLACE]->(i2);',
        'MATCH (i1:Ingredient {name: "Şeker"}), (i2:Ingredient {name: "Bal"}) MERGE (i1)-[:CAN_REPLACE]->(i2);'
    ]
    print("Başlangıç şeması Neo4j'e yazılıyor...")
    for stmt in initial_cypher_statements:
        graph.query(stmt)
    print("Başlangıç şeması yazıldı.")

In [7]:
# --- 4. Yemek Detaylarını Embedding ile Neo4j'e Basma ---
def populate_yemek_detaylari(graph, embeddings_client):
    """Yemek detaylarını alır, embedding oluşturur ve Neo4j'e yazar."""
    yemek_detaylari_data = [
        {
            "detay_id": "sutlac_detay_001", "tarif_adi": "Sütlaç",
            "hazirlanis_asamlari_metni": "1. Pirinci bol suda yıkayıp süzün...",
            "ipuclari_ve_puf_noktalari_metni": "Daha kremamsı bir kıvam için...",
            "sunum_onerileri_metni": "Soğuk servis yapılması önerilir...",
            "koken_hikayesi_metni": "Sütlaç, 'sütlü aş' kelimelerinin birleşiminden...",
            "varyasyonlar_metni": "Fırın Sütlaç: Kaselere paylaştırılan...",
            "besin_degerleri_metni": "Bir kase (yaklaşık 200g) sütlaç...",
            "gömme_icin_tam_aciklama": "Sütlaç Tarifi Detayları: Hazırlanışı: Pirinci yıkayıp süzün. Gerekirse yarım saat kadar ılık suda bekletebilirsiniz. Sütü geniş bir tencereye alın. Yıkanmış pirinci ekleyin. Orta ateşte, pirinçler iyice yumuşayana ve süt kıvam alana kadar yaklaşık 30-40 dakika boyunca ara sıra karıştırarak pişirin. Pirinçlerin dibe yapışmamasına özen gösterin. Pirinçler yumuşayınca toz şekeri ilave edin. Şeker eriyene ve karışım bir taşım daha kaynayana kadar (yaklaşık 5-10 dakika) karıştırarak pişirmeye devam edin. Vanilini ekleyin, karıştırın ve ocağın altını kapatın. Hazırladığınız sütlacı ısıya dayanıklı kaselere veya güveç kaplarına paylaştırın. Oda sıcaklığına geldikten sonra buzdolabına kaldırın ve en az 2-3 saat soğutun. Servis yapmadan önce üzerini bol tarçınla süsleyin. İpuçları: Daha kremamsı bir kıvam için bir miktar krema eklenebilir veya pirinç unu/nişasta ile kıvamı bağlanabilir. Sütlacın taşmasını önlemek için tencerenin ağzına tahta bir kaşık koyabilirsiniz. Şeker miktarını damak zevkinize göre ayarlayabilirsiniz. Sunum: Soğuk servis yapılması önerilir. Üzerine tarçın serpilebileceği gibi, dövülmüş fındık, ceviz veya antep fıstığı ile de süslenebilir. Yanında taze mevsim meyveleriyle de farklı bir sunum yapılabilir. Kökeni: Sütlaç, 'sütlü aş' kelimelerinin birleşiminden türemiş olup, Türk mutfağının en eski ve en sevilen tatlılarından biridir. Osmanlı saray mutfağında da önemli bir yere sahip olmuştur. Varyasyonlar: Fırın Sütlaç: Kaselere paylaştırılan sütlaçların üzerine çırpılmış yumurta sarısı sürülerek fırında üstleri kızarana kadar pişirilir. Damla Sakızlı Sütlaç: Pişirme aşamasında birkaç parça damla sakızı eklenerek farklı bir aroma katılabilir. Besin Değerleri: Bir kase (yaklaşık 200g) sütlaç ortalama 250-300 kalori içerebilir. Bu değer kullanılan şeker ve süt tipine göre değişiklik gösterebilir.",
        },
        {
            "detay_id": "kek_detay_001", "tarif_adi": "Kek",
            "hazirlanis_asamlari_metni": "1. Fırını 180°C'ye ayarlayın...",
            "ipuclari_ve_puf_noktalari_metni": "Malzemelerin oda sıcaklığında olması...",
            "sunum_onerileri_metni": "Dilimleyerek servis yapın...",
            "koken_hikayesi_metni": "Kek, un, şeker, yumurta gibi temel malzemelerle...",
            "varyasyonlar_metni": "Meyveli kek, havuçlu kek...",
            "besin_degerleri_metni": "Bir dilim sade kek ortalama 200-250 kalori...",
            "gömme_icin_tam_aciklama": "Kek Tarifi Detayları: Hazırlanışı: Fırını 180°C'ye ayarlayın ve kek kalıbını yağlayıp unlayın. Yumurtaları ve şekeri bir kapta köpürene kadar çırpın. Sıvı yağ (veya eritilmiş tereyağı) ve sütü ekleyip karıştırın. Un, kabartma tozu ve vanilini eleyerek karışıma azar azar ekleyin ve düşük devirde veya spatula ile homojen bir kıvam alana kadar karıştırın. Karışımı hazırladığınız kek kalıbına dökün. Önceden ısıtılmış fırında yaklaşık 30-40 dakika veya kürdan temiz çıkana kadar pişirin. Fırından çıkan keki 10 dakika dinlendirdikten sonra kalıptan çıkarın ve soğumaya bırakın. İpuçları: Malzemelerin oda sıcaklığında olması kekin daha iyi kabarmasını sağlar. Unu elemek topaklanmayı önler. Fırının kapağını ilk 20-25 dakika açmayın. Sunum: Dilimleyerek servis yapın. Üzerine pudra şekeri serpebilir, yanında çay veya kahve ile ikram edebilirsiniz. Kökeni: Kek, un, şeker, yumurta gibi temel malzemelerle hazırlanan, dünya genelinde popüler bir fırın ürünüdür. Varyasyonlar: Meyveli kek, havuçlu kek, ıslak kek, kakaolu kek gibi birçok çeşidi yapılabilir. Besin Değerleri: Bir dilim sade kek (yaklaşık 60g) ortalama 200-250 kalori içerebilir.",
        },
        {
            "detay_id": "cheesecake_detay_001", "tarif_adi": "Cheesecake",
            "hazirlanis_asamlari_metni": "1. Taban için: Bisküvileri rondodan geçirin...",
            "ipuclari_ve_puf_noktalari_metni": "Peynirli karışımı fazla çırpmayın...",
            "sunum_onerileri_metni": "Dilimlenerek soğuk servis yapılır...",
            "koken_hikayesi_metni": "Modern krem peynirli cheesecake Amerika'da...",
            "varyasyonlar_metni": "Limonlu cheesecake, çikolatalı cheesecake...",
            "besin_degerleri_metni": "Bir dilim cheesecake ortalama 300-400 kalori...",
            "gömme_icin_tam_aciklama": "Cheesecake Tarifi Detayları: Hazırlanışı: Taban için: Bisküvileri rondodan geçirin, eritilmiş tereyağı ile karıştırın ve kalıbın tabanına yayın. Buzdolabında bekletin. Peynirli dolgu için: Labne peyniri, krema ve şekeri çırpın. Yumurtaları teker teker ekleyin. Vanilin ekleyip karıştırın. Karışımı tabanın üzerine dökün. Önceden 160°C'ye ısıtılmış fırında yaklaşık 50-60 dakika pişirin. Fırını kapatıp fırında dinlendirin. Oda sıcaklığına geldikten sonra buzdolabında en az 4-6 saat dinlendirin. Üzerini meyve sosu ile süsleyerek servis yapın. İpuçları: Peynirli karışımı fazla çırpmayın. Fırında su dolu bir kap kullanmak çatlamaları önler. Yavaş soğutmak önemlidir. Sunum: Dilimlenerek soğuk servis yapılır. Üzerine frambuaz, limon veya karamel sosu çok yakışır. Kökeni: Modern krem peynirli cheesecake Amerika'da popülerleşmiştir, kökenleri antik Yunanistan'a dayanır. Varyasyonlar: Limonlu cheesecake, çikolatalı cheesecake, San Sebastian cheesecake gibi türleri vardır. Besin Değerleri: Bir dilim cheesecake (yaklaşık 100g) ortalama 300-400 kalori içerebilir.",
        },
        {
            "detay_id": "truffle_detay_001", "tarif_adi": "Çikolata Truffle",
            "hazirlanis_asamlari_metni": "1. Kremayı ısıtın, kaynama noktasına gelmeden...",
            "ipuclari_ve_puf_noktalari_metni": "Kaliteli çikolata kullanın...",
            "sunum_onerileri_metni": "Küçük kağıt kapsüllerde servis edin...",
            "koken_hikayesi_metni": "Adını değerli trüf mantarına benzeyen...",
            "varyasyonlar_metni": "Sütlü veya beyaz çikolatalı yapılabilir...",
            "besin_degerleri_metni": "Bir adet çikolata truffle ortalama 70-90 kalori...",
            "gömme_icin_tam_aciklama": "Çikolata Truffle Tarifi Detayları: Hazırlanışı: Kremayı ısıtın, kaynama noktasına gelmeden ocaktan alın. Doğranmış bitter çikolatanın üzerine sıcak kremayı dökün, eriyene kadar bekletin. Tereyağını ekleyip pürüzsüz bir ganaj elde edene kadar karıştırın. Karışımı buzdolabında katılaşana kadar bekletin. Katılaşan ganajdan toplar yapın ve kakao, pudra şekeri veya rendelenmiş çikolataya bulayın. İpuçları: Kaliteli çikolata kullanın. Ganajı yavaşça karıştırın. Ellerinizi soğutarak çalışın. Sunum: Küçük kağıt kapsüllerde veya şık bir tabakta servis edin. Kahve eşliğinde ikram edilebilir. Kökeni: Adını değerli trüf mantarına benzeyen görünümünden alır. İlk olarak Fransa'da ortaya çıktığı düşünülür. Varyasyonlar: Sütlü veya beyaz çikolatalı yapılabilir. İçine farklı aromalar veya kuru yemiş eklenebilir. Besin Değerleri: Bir adet çikolata truffle (yaklaşık 15g) ortalama 70-90 kalori içerebilir.",
        },
        {
            "detay_id": "magnolia_detay_001", "tarif_adi": "Magnolia",
            "hazirlanis_asamlari_metni": "1. Muhallebi için: Süt, un, nişasta...",
            "ipuclari_ve_puf_noktalari_metni": "Muhallebiyi sürekli karıştırarak pişirin...",
            "sunum_onerileri_metni": "Tek kişilik kuplarda servis yapın...",
            "koken_hikayesi_metni": "Amerika'daki Magnolia Bakery ile popülerleşmiş...",
            "varyasyonlar_metni": "Çilekli, muzlu, çikolatalı muhallebi ile...",
            "besin_degerleri_metni": "Bir porsiyon magnolia ortalama 300-350 kalori...",
            "gömme_icin_tam_aciklama": "Magnolia Tarifi Detayları: Hazırlanışı: Muhallebi için: Süt, un, nişasta, şeker ve yumurta sarısını çırpıp pişirin. Ocaktan alınca vanilin ve tereyağı/krema ekleyin. Bisküvileri rondodan geçirin. Kupların tabanına bisküvi, üzerine muhallebi, aralara meyve (çilek/muz) şeklinde katlar yapın. En üste bisküvi veya muhallebi ile bitirin. Buzdolabında en az 2-3 saat soğutun. İpuçları: Muhallebiyi sürekli karıştırarak pişirin. Piştikten sonra mikserle çırpmak pürüzsüzlüğü artırır. Bebe bisküvisi veya yulaflı bisküvi kullanılabilir. Sunum: Tek kişilik kuplarda servis yapın. Üzerini taze meyveler ve bisküvi kırıntıları ile süsleyin. Kökeni: Amerika'daki Magnolia Bakery ile popülerleşmiş, katmanlı, bisküvili ve muhallebili bir tatlıdır. Varyasyonlar: Çilekli, muzlu, çikolatalı muhallebi ile veya farklı bisküvi türleriyle yapılabilir. Besin Değerleri: Bir porsiyon magnolia (yaklaşık 200g) ortalama 300-350 kalori içerebilir.",
        }
    ]

    print("Yemek detayları Neo4j'e yazılıyor ve embedding oluşturuluyor...")
    for detay_data in yemek_detaylari_data:
        text_to_embed = detay_data["gömme_icin_tam_aciklama"]
        embedding_vector = embeddings_client.embed_query(text_to_embed)

        # Cypher sorgusu YemekDetayi node'unu oluşturur/günceller ve Recipe'ye bağlar
        query = """
        MERGE (yd:YemekDetayi {detay_id: $detay_id})
        ON CREATE SET
            yd.tarif_adi = $tarif_adi,
            yd.hazirlanis_asamlari_metni = $hazirlanis_asamlari_metni,
            yd.ipuclari_ve_puf_noktalari_metni = $ipuclari_ve_puf_noktalari_metni,
            yd.sunum_onerileri_metni = $sunum_onerileri_metni,
            yd.koken_hikayesi_metni = $koken_hikayesi_metni,
            yd.varyasyonlar_metni = $varyasyonlar_metni,
            yd.besin_degerleri_metni = $besin_degerleri_metni,
            yd.gömme_icin_tam_aciklama = $gömme_icin_tam_aciklama,
            yd.embedding = $embedding_vector
        ON MATCH SET // Eğer node zaten varsa sadece embedding'i güncelleyebiliriz veya diğerlerini de
            yd.tarif_adi = $tarif_adi,
            yd.hazirlanis_asamlari_metni = $hazirlanis_asamlari_metni,
            yd.ipuclari_ve_puf_noktalari_metni = $ipuclari_ve_puf_noktalari_metni,
            yd.sunum_onerileri_metni = $sunum_onerileri_metni,
            yd.koken_hikayesi_metni = $koken_hikayesi_metni,
            yd.varyasyonlar_metni = $varyasyonlar_metni,
            yd.besin_degerleri_metni = $besin_degerleri_metni,
            yd.gömme_icin_tam_aciklama = $gömme_icin_tam_aciklama,
            yd.embedding = $embedding_vector
        WITH yd
        MATCH (r:Recipe {name: $tarif_adi})
        MERGE (r)-[:DETAYINA_SAHIPTIR]->(yd)
        """
        
        params = {**detay_data, "embedding_vector": embedding_vector}
        graph.query(query, params=params)
        print(f"{detay_data['tarif_adi']} için detaylar ve embedding eklendi.")
    print("Tüm yemek detayları ve embeddingleri Neo4j'e yazıldı.")

    # Embedding içeren nodelarda vektör indeksi oluşturma (opsiyonel ama performans için iyi)
    # Bu, Neo4jVector.from_existing_graph kullanırken otomatik olabilir veya manuel oluşturulabilir.
    # Şimdilik manuel bir indeks oluşturma sorgusu ekleyelim:
    index_query = """
    CREATE VECTOR INDEX `yemekDetayiEmbeddings` IF NOT EXISTS
    FOR (yd:YemekDetayi) ON (yd.embedding)
    OPTIONS {indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
    }}
    """
    try:
        graph.query(index_query)
        print("YemekDetayi embedding'leri için vektör indeksi oluşturuldu/kontrol edildi.")
    except Exception as e:
        print(f"Vektör indeksi oluşturulurken hata: {e} (Belki zaten var veya Neo4j sürümü desteklemiyor)")

In [8]:
# --- 5. LLM ile Cypher Sorgusu Üretme ---
def generate_cypher_with_llm(graph, llm, user_question):
    """Verilen şema ve soru ile LLM kullanarak Cypher sorgusu üretir."""
    
    # Graf şemasını al (Langchain Neo4jGraph bunu otomatik yapar)
    # graph.refresh_schema() # Gerekirse şemayı yenile
    
    # Langchain'in GraphCypherQAChain'ini kullanalım
    # Bu zincir hem Cypher üretir hem de sonucu LLM ile doğal dile çevirir.
    # Biz sadece üretilen Cypher'ı almak istiyoruz.
    
    CYPHER_GENERATION_TEMPLATE = """
    Task: Generate Cypher query to query a graph database.
    Instructions:
    Use only the provided schema for node labels, relationship types, and properties.
    Do not use any other node labels, relationship types, or properties that are not provided.
    If the question involves finding recipes based on similarity of their details (like preparation, tips etc.),
    you can use vector similarity search on the `YemekDetayi` node's `embedding` property.
    The vector index is on `YemekDetayi(embedding)`.
    To perform a similarity search, you can use `db.index.vector.queryNodes`.
    For example:
    'CALL db.index.vector.queryNodes('yemekDetayiEmbeddings', 10, $embeddingParam) YIELD node AS similarDetay, score'
    Where 'yemekDetayiEmbeddings' is the index name, 10 is the number of similar items, and $embeddingParam is the embedding of the user's query text.
    Then you can match the Recipe: `MATCH (recipe:Recipe)-[:DETAYINA_SAHIPTIR]->(similarDetay) RETURN recipe.name, score`

    Schema:
    {schema}

    Question: {question}
    Cypher Query:
    """
    
    cypher_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CYPHER_GENERATION_TEMPLATE),
        ]
    )

    # GraphCypherQAChain en son langchain_neo4j paketi içindedir
    # `allow_dangerous_requests=True` güvenlik notunu dikkate alarak kullanın.
    # Eğer sadece sorgu üretmek ve çalıştırmadan görmek istiyorsanız,
    # zinciri buna göre ayarlamanız veya manuel bir LLM çağrısı yapmanız gerekebilir.
    # GraphCypherQAChain genellikle tam bir Q&A akışı içindir.
    
    # Daha doğrudan bir yaklaşım: LLM'e şemayı ve soruyu verip sadece sorgu üretmesini istemek.
    chain_for_cypher_generation = cypher_prompt | llm
    
    # Kullanıcı sorusunun embedding'ini de oluşturup parametre olarak LLM'e verebiliriz,
    # eğer benzerlik araması yapmasını istiyorsak.
    # Ancak bu örnekte, LLM'in genel sorgu üretme yeteneğini test ediyoruz.
    
    response = chain_for_cypher_generation.invoke({
        "schema": graph.schema, # Neo4jGraph objesinden şemayı alır
        "question": user_question
    })
    
    generated_cypher_query = response.content.strip()
    
    # Alternatif olarak GraphCypherQAChain kullanarak:
    # qa_chain = GraphCypherQAChain.from_llm(
    #     llm, # Cypher üreten ve cevabı oluşturan LLM için
    #     graph=graph,
    #     verbose=True,
    #     return_intermediate_steps=True, # Üretilen Cypher'ı görmek için
    #     # cypher_prompt=custom_prompt, # İsteğe bağlı özel prompt
    #     # qa_prompt=custom_qa_prompt
    # )
    # result = qa_chain.invoke({"query": user_question})
    # generated_cypher_query = result["intermediate_steps"][0]["query"] # Genellikle burada olur

    print(f"\nKullanıcı Sorusu: {user_question}")
    print(f"Üretilen Cypher Sorgusu:\n{generated_cypher_query}")
    return generated_cypher_query

In [13]:
# --- Ana Çalıştırma Bloğu ---
if __name__ == "__main__":
    neo4j_uri_val, neo4j_username_val, neo4j_password_val = get_api_keys()
    
    graph_client, embeddings_model, llm_model = setup_clients(
        neo4j_uri_val, neo4j_username_val, neo4j_password_val
    )

    # Adım 3: Başlangıç şemasını bas
    # Bu adımı her seferinde çalıştırmak yerine, veritabanı boşsa bir kere çalıştırın.
    # Örneğin, veritabanında belirli bir node'un varlığını kontrol ederek.
    # Bu demo için her seferinde çalıştırıyoruz (MERGE sayesinde sorun olmaz).
    populate_initial_schema(graph_client)
    
    # Adım 4: Yemek detaylarını embedding ile bas
    populate_yemek_detaylari(graph_client, embeddings_model)

    # Adım 5: LLM ile Cypher sorgusu üret
    print("\n--- LLM ile Cypher Sorgu Üretme ---")
    graph_client.refresh_schema() # Yeni eklenen node ve ilişkiler için şemayı güncelle
    
    # Örnek sorular:
    # "Süt içeren ve zorluk derecesi kolay olan tarifler nelerdir?"
    # "Fırınlama tekniği kullanılan ve tereyağı içeren tarifler hangileri?"
    # "Bana kremamsı lezzete sahip tüm tarifleri ve hazırlanma sürelerini göster."
    # "Sütlaç tarifinin detaylı hazırlanışını ve püf noktalarını anlatır mısın?" (Bu RAG gerektirir, sadece Cypher değil)
    # "Pirinç ile yapılan tatlılar nelerdir?"
    
    # Basit bir Cypher üretme sorusu:
    soru1 = "Şeker ve süt içeren tüm tariflerin isimlerini ve zorluk derecelerini listele."
    generate_cypher_with_llm(graph_client, llm_model, soru1)

    # Embedding kullanarak benzerlik araması için bir soru (LLM'e ipucu verdik)
    # Bu sorunun embedding'ini de alıp LLM'e parametre olarak vermek daha iyi sonuç verir,
    # ama şimdilik LLM'in prompttaki talimatlara göre sorgu üretip üretmediğini görelim.
    soru2 = "Sütlaç tarifine benzer hazırlanan başka tarifler var mı?" 
    # Bu soru için, "Sütlaç tarifinin gömme_icin_tam_aciklama metninin embedding'ini alıp
    # bunu $embeddingParam olarak kullan" şeklinde LLM'e daha detaylı talimat gerekebilir
    # veya uygulama katmanında Sütlaç'ın embedding'i çekilip sorguya parametre olarak verilir.
    # Şimdilik LLM'in `db.index.vector.queryNodes` kullanıp kullanmadığına bakalım.
    
    # Soru 2 için embedding parametresini manuel hazırlayalım:
    sutlac_embedding_text = "Sütlaç Tarifi Detayları: Hazırlanışı: Pirinci yıkayıp süzün..." # Sütlaç'ın tam açıklama metni
    sutlac_query_embedding = embeddings_model.embed_query(sutlac_embedding_text)

    # LLM'e bu embedding'i kullanarak bir sorgu üretmesini isteyelim
    # (Bu kısım daha karmaşık bir prompt mühendisliği gerektirebilir)
    # Şimdilik bu adımı geçip, LLM'in genel şema bilgisiyle nasıl sorgu ürettiğine odaklanalım.
    # Daha gelişmiş RAG senaryolarında bu tür bir akış kullanılır.

    print("\nNeo4j'deki grafiği Neo4j Browser üzerinden inceleyebilirsiniz.")
    print("Örnek sorgular çalıştırmak için Neo4j Browser'ı kullanın veya bu script'i genişletin.")


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resu

Başlangıç şeması Neo4j'e yazılıyor...
Başlangıç şeması yazıldı.
Yemek detayları Neo4j'e yazılıyor ve embedding oluşturuluyor...


AttributeError: 'SentenceTransformer' object has no attribute 'embed_query'